In [79]:
import pandas as pd
import re
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from sklearn.svm import SVC
from collections import Counter
from sklearn.metrics import confusion_matrix

In [80]:
df = pd.read_csv("fake_and_real_news.csv")
df

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real
...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake
9896,Trump consults Republican senators on Fed chie...,Real
9897,Trump lawyers say judge lacks jurisdiction for...,Real
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake


In [81]:
df.tail(100)

,Text,label
9800,Republican Party backs Senate candidate Moore:...,Real
9801,Republican senators want review of 'too big to...,Real
9802,Mexican President Tells Trump One More Time T...,Fake
9803,Black Military Veteran Denied Meal Because Wh...,Fake
9804,WATCH: ‘The View’ Hosts Tell Trump To Get The...,Fake
...,...,...
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake
9896,Trump consults Republican senators on Fed chie...,Real
9897,Trump lawyers say judge lacks jurisdiction for...,Real
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake


In [82]:
df.shape

(9900, 2)

In [83]:
# Number of rows labelled as fake news

sum_rows_fake = df[df['label']=='Fake'].shape[0]

print(sum_rows_fake)

5000


In [84]:
sum_rows_real = df[df['label']=='Real'].shape[0]
print(sum_rows_real)


4900


In [85]:
# Missing values count in dataset
missing_values = df["Text"].isnull().sum()
print(str(missing_values) + " missing values")

0 missing values


In [86]:
# number of duplicate values in text column
num_duplicates = df["Text"].duplicated().sum()

print(str(num_duplicates) + " number of duplicate rows")

35 number of duplicate rows


In [87]:
df = df.drop_duplicates(subset="Text", keep="first")
df['Text'] = df['Text'].astype(str)

num_duplicates = df["Text"].duplicated().sum()
print(str(num_duplicates) + " number of duplicate rows")

0 number of duplicate rows


/var/folders/ww/d_0mlqy97tgfbnd0s2t91g_m0000gn/T/ipykernel_6275/1174393082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'] = df['Text'].astype(str)


In [88]:
df['processed_text'] = df['Text'].str.lower()
df['processed_text'] = df['processed_text'].astype(str)

df.head()

/var/folders/ww/d_0mlqy97tgfbnd0s2t91g_m0000gn/T/ipykernel_6275/472969064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['Text'].str.lower()
/var/folders/ww/d_0mlqy97tgfbnd0s2t91g_m0000gn/T/ipykernel_6275/472969064.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].astype(str)


,Text,label,processed_text
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,top trump surrogate brutally stabs him in the...
1,U.S. conservative leader optimistic of common ...,Real,u.s. conservative leader optimistic of common ...
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,"trump proposes u.s. tax overhaul, stirs concer..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,court forces ohio to allow millions of illega...
4,Democrats say Trump agrees to work on immigrat...,Real,democrats say trump agrees to work on immigrat...


In [89]:
#preprocessing

# remove punctuation
df['processed_text'] = df['processed_text'].str.replace(r'[^\w\s]', '', regex=True)

#remove special characters
df['processed_text'] = df['processed_text'].str.replace(r'[^A-Za-z0-9 ]+', '', regex=True)

#remove numbers
df['processed_text'] = df['processed_text'].str.replace(r'\d+', '', regex=True)
df.tail()

/var/folders/ww/d_0mlqy97tgfbnd0s2t91g_m0000gn/T/ipykernel_6275/484347262.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].str.replace(r'[^\w\s]', '', regex=True)
/var/folders/ww/d_0mlqy97tgfbnd0s2t91g_m0000gn/T/ipykernel_6275/484347262.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].str.replace(r'[^A-Za-z0-9 ]+', '', regex=True)
/var/folders/ww/d_0mlqy97tgfbnd0s2t91g_m0000gn/T/ipykernel_6275/484347262.py:10: Sett

,Text,label,processed_text
9895,Wikileaks Admits To Screwing Up IMMENSELY Wit...,Fake,wikileaks admits to screwing up immensely wit...
9896,Trump consults Republican senators on Fed chie...,Real,trump consults republican senators on fed chie...
9897,Trump lawyers say judge lacks jurisdiction for...,Real,trump lawyers say judge lacks jurisdiction for...
9898,WATCH: Right-Wing Pastor Falsely Credits Trum...,Fake,watch rightwing pastor falsely credits trump ...
9899,Sean Spicer HILARIOUSLY Branded As Chickensh*...,Fake,sean spicer hilariously branded as chickensht...


In [90]:
# tokenize words
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

df['processed_text'] = df['processed_text'].astype(str)
df['processed_text'] = df['processed_text'].apply(word_tokenize)

df.head(50)

[nltk_data] Downloading package punkt_tab to /Users/ahad/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/var/folders/ww/d_0mlqy97tgfbnd0s2t91g_m0000gn/T/ipykernel_6275/2168915768.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].astype(str)
/var/folders/ww/d_0mlqy97tgfbnd0s2t91g_m0000gn/T/ipykernel_6275/2168915768.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].apply(word_tokenize)


,Text,label,processed_text
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,"[top, trump, surrogate, brutally, stabs, him, ..."
1,U.S. conservative leader optimistic of common ...,Real,"[us, conservative, leader, optimistic, of, com..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,"[trump, proposes, us, tax, overhaul, stirs, co..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,"[court, forces, ohio, to, allow, millions, of,..."
4,Democrats say Trump agrees to work on immigrat...,Real,"[democrats, say, trump, agrees, to, work, on, ..."
5,France says pressure needed to stop North Kore...,Real,"[france, says, pressure, needed, to, stop, nor..."
6,"Trump on Twitter (August 8): Opioid crisis, No...",Real,"[trump, on, twitter, august, opioid, crisis, n..."
7,BUSTED: Trump Supporter Used Poll Watcher Cre...,Fake,"[busted, trump, supporter, used, poll, watcher..."
8,Fatal Niger operation sparks calls for public ...,Real,"[fatal, niger, operation, sparks, calls, for, ..."
9,Trump says he has 'great heart' for immigrant ...,Real,"[trump, says, he, has, great, heart, for, immi..."


In [91]:
# stopwords removal
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

df['processed_text'] = df['processed_text'].apply(
    lambda tokens: [word for word in tokens if word not in stop_words])

df.head()

[nltk_data] Downloading package stopwords to /Users/ahad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/ww/d_0mlqy97tgfbnd0s2t91g_m0000gn/T/ipykernel_6275/138707364.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['processed_text'].apply(


,Text,label,processed_text
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,"[top, trump, surrogate, brutally, stabs, back,..."
1,U.S. conservative leader optimistic of common ...,Real,"[us, conservative, leader, optimistic, common,..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,"[trump, proposes, us, tax, overhaul, stirs, co..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,"[court, forces, ohio, allow, millions, illegal..."
4,Democrats say Trump agrees to work on immigrat...,Real,"[democrats, say, trump, agrees, work, immigrat..."


In [92]:
#lemmatization

# we chose lemmatization over stemming as the processing time was worth the wait for higher accuracy a

nlp = spacy.load("en_core_web_sm")

df["processed_text"] = df["processed_text"].apply(
    lambda tokens: [token.lemma_ for token in nlp(" ".join(tokens))]  
)

df.head()

/var/folders/ww/d_0mlqy97tgfbnd0s2t91g_m0000gn/T/ipykernel_6275/1624024136.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["processed_text"] = df["processed_text"].apply(


,Text,label,processed_text
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,"[top, trump, surrogate, brutally, stab, back, ..."
1,U.S. conservative leader optimistic of common ...,Real,"[us, conservative, leader, optimistic, common,..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,"[trump, propose, we, tax, overhaul, stir, conc..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,"[court, force, ohio, allow, million, illegally..."
4,Democrats say Trump agrees to work on immigrat...,Real,"[democrats, say, trump, agree, work, immigrati..."
